## Pregunta: T4 - ¿La arquitectura, sección o mantenedor influye en el número de dependencias de un paquete?


### 1. Importaciones

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from scipy import stats
import statsmodels.api as sm

from IPython.display import display, Markdown

### 2. Lectura

In [2]:
df = pd.read_csv("../../datawarehouse/processed-ubuntu-packages.csv")

### 3. Preprocesado

In [45]:
df = pd.read_csv("../../datawarehouse/processed-ubuntu-packages.csv")
# Obtener unicamente Architecture, Section, Maintainer, Package y Depends
df = df[['Architecture', 'Section', 'Maintainer', 'Package', 'Depends']]

# Convertimos Depends a lista
df['Depends'] = df['Depends'].apply(lambda x: str(x).split(','))

# Calculamos el número de dependencias de cada paquete y agregarlo como una nueva columna al dataframe
df['Num_Depends'] = df['Depends'].apply(lambda x: len(x))

df.head()

,Architecture,Section,Maintainer,Package,Depends,Num_Depends
0,amd64,admin,Ubuntu,acct,"[libc6 (>= 2.14), lsb-base]",2
1,amd64,utils,Ubuntu,acl,"[libacl1 (= 2.2.52-3build1), libc6 (>= 2.14)]",2
2,amd64,admin,Ubuntu,acpi-support,[acpid (>= 1.0.4-1ubuntu4)],1
3,amd64,admin,Ubuntu,acpid,"[libc6 (>= 2.15), init-system-helpers (>= 1.1...",4
4,all,admin,Ubuntu,adduser,"[passwd, debconf (>= 0.5) | debconf-2.0]",2


### 4. Análisis

#### Arquitectura

In [16]:
# Veremos los distintos valores que toma la columna Architecture
print("Architecture: ", df['Architecture'].unique())

Architecture:  ['amd64' 'all']


In [26]:
# Para calcular la correlación entre las variables, necesitamos convertir las variables categóricas a numéricas
# Cambiaremos amd64 a 0 y all a 1
df_arch = df['Architecture'].apply(lambda x: 0 if x == 'amd64' else 1)

# Prueba de hipótesis para la correlación entre Num_Depends y Architecture
corr_architecture, p_architecture = pearsonr(df['Num_Depends'], df_arch)
print("Correlación entre Num_Depends y Architecture: ", corr_architecture)
print("Valor p para la correlación entre Num_Depends y Architecture: ", p_architecture)

Correlación entre Num_Depends y Architecture:  -0.11829892015323715
Valor p para la correlación entre Num_Depends y Architecture:  0.0


El resultado indica que hay una correlación negativa débil entre el número de dependencias de un paquete de Linux y la arquitectura que acepta. Como se ha asignado un valor numérico más alto (1) a la etiqueta 'all' que a la etiqueta 'amd64' (0), el hecho de que la correlación sea negativa indica que los paquetes con un número mayor de dependencias son menos propensos a ser compatibles con la arquitectura 'all'.

El valor p de 0.0 indica que la correlación es estadísticamente significativa, lo que sugiere que la relación entre el número de dependencias y la arquitectura aceptada no es simplemente una coincidencia aleatoria y puede tener implicaciones prácticas en la selección y compatibilidad de los paquetes.

In [44]:
# Importamos la función para calcular la prueba de hipótesis de igualdad de varianzas
from scipy.stats import f_oneway

df_amd64 = df[df['Architecture'] == 'amd64']
df_all = df[df['Architecture'] == 'all']


f_stat, p_value = f_oneway(df_amd64['Num_Depends'], df_all['Num_Depends'])
print("Valor f para la prueba de hipótesis de igualdad de varianzas: ", f_stat)
print("Valor p para la prueba de hipótesis de igualdad de varianzas: ", p_value)


Valor f para la prueba de hipótesis de igualdad de varianzas:  4254.09026135063
Valor p para la prueba de hipótesis de igualdad de varianzas:  0.0


El valor F es una estadística de prueba que se utiliza para comparar la varianza de dos muestras. En este caso, el valor F es 4254.09026135063, lo que indica que la varianza del número de dependencias es significativamente diferente entre los dos grupos.

El valor p es la probabilidad de obtener los resultados observados si la hipótesis nula es verdadera. En este caso, la hipótesis nula es que la varianza del número de dependencias es igual en los dos grupos. El valor p que has obtenido es 0.0, lo que indica que la probabilidad de obtener los resultados observados si la hipótesis nula es verdadera es extremadamente baja. Por lo tanto, podemos concluir que hay una diferencia significativa en la varianza del número de dependencias entre los paquetes con arquitectura "amd64" y los paquetes con arquitectura "all".

#### Sección

In [43]:
# Veremos los distintos valores que toma la columna Section
print("Número de secciones distintas: ", df['Section'].nunique())

Número de secciones distintas:  150


In [53]:
from scipy.stats import levene
# Test estadístico para ver si la distribución de Num_Depends es la misma para cada sección
# Para esto, necesitamos dividir el dataframe en subdataframes, uno por cada sección
# Para esto, agruparemos el dataframe por Section y obtendremos los grupos
df_grouped = df.groupby('Section')


# Para cada grupo, calcularemos el valor f y el valor p
# Lo almacenaremos en un diccionario
dict_f = {}
for name, group in df_grouped:
    f_stat, p_value = levene(group['Num_Depends'], df['Num_Depends'])
    # almacena los valores f y p en un diccionario
    dict_f[name] = f_stat

# Veremos si hay mucha diferencia con el valor f para la prueba de hipótesis de igualdad de varianzas entre el primer y ultimo grupo
# Ordenamos el diccionario por valor f
dict_f = sorted(dict_f.items(), key=lambda x: x[1], reverse=True)
print("Sección con mayor diferencia: ", dict_f[0])
print("Sección con menor diferencia: ", dict_f[-1])

# Media de las diferencias
mean_diff = np.mean([x[1] for x in dict_f])
print("Media de las diferencias: ", mean_diff)




Sección con mayor diferencia:  ('universe/kde', 3677.3298140720203)
Sección con menor diferencia:  ('universe/gnustep', 9.931416478491671e-05)
Media de las diferencias:  113.05576929742968


La prueba de hipótesis de igualdad de varianzas compara la varianza de dos o más grupos. Si el valor p de la prueba es menor que el nivel de significancia establecido (generalmente 0.05), se rechaza la hipótesis nula de que las varianzas son iguales y se concluye que al menos una varianza es significativamente diferente de las demás.

En este caso, los resultados muestran que hay una gran diferencia en las varianzas de Num_Depends entre las secciones 'universe/kde' y 'universe/gnustep', lo que se indica por la gran diferencia en el valor F (3677.33 frente a 0.0001) entre estas dos secciones.

Además, la media de las diferencias entre las varianzas de todas las secciones es 113.06, lo que sugiere que en general hay diferencias significativas entre las varianzas de las secciones.

#### Mantenedor

In [58]:
# Un test estadísitico que podemos usar para analizar cómo influyen los Mantainers en el número de dependencias es el test de ANOVA

groups = df.groupby('Maintainer')['Num_Depends'].apply(list)
anova_test = f_oneway(*groups)

print("El valor estadístico del test es:", anova_test.statistic)
print("El valor p del test es:", anova_test.pvalue)


El valor estadístico del test es: 43.74539776407028
El valor p del test es: 0.0


El valor estadístico del test ANOVA que hemos obtenido es de 43.75 y el valor p es de 0.0. Esto significa que existe una diferencia estadísticamente significativa en el número promedio de dependencias entre los diferentes grupos de Maintainer.

En otras palabras, la variable categórica Maintainer tiene un efecto significativo en el número promedio de dependencias de un paquete. Por lo tanto, es probable que los distintos mantenedores del paquete tengan diferentes prácticas y/o estándares al momento de definir las dependencias de sus paquetes.